### Code Carbon Notebook

Within this jupyter notebook I want to demonstrate the utility of Code Carbon as a Python package to track CO2 emissions during machine learning model training.
... more bla bla ... 

Setup environment

In [1]:
!conda create -n codecarbon_env python=3.11.5 -y

Retrieving notices: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\Christopher.Danz\anaconda3\envs\codecarbon_env

  added / updated specs:
    - python=3.11.5


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-3.0.12             |       h2bbff1b_0         7.4 MB
    pip-23.3.1                 |  py311haa95532_0         3.5 MB
    xz-5.4.5                   |       h8cc25b3_0         593 KB
    ------------------------------------------------------------
                                           Total:        11.5 MB

The following NEW packages will be INSTALLED:

  bzip2              pkgs/main/win-64::bzip2-1.0.8-he774522_0 
  ca-certificates    pkgs/main/win-64::ca-certificates-2023.08.22-haa95532_0 
  libffi             pkgs/main/win-64::libffi-3.4.4-hd77b12b_0 
  openssl            pkgs/main/win-64::openssl-3.0



==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0




In [3]:
!conda activate codecarbon_env

In [4]:
!pip install codecarbon
!pip install sklearn
!pip install tensorflow

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl (1

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#### Data Preparation

In [8]:
# import the dataset
data = load_iris()
X, y = data.data, data.target

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Simple Model Training and Emission Tracking 

Typical problems you run into:
- NVML Error: This means that your GPU is currently not supported by NVIDIA Management Library (NVML) \
    Options to solve it: 
    1) checking in NVIDIA documentation if your device-specific GPU is supported or not (if not, disable GPU tracking with CodeCarbon or update GPU driver)
    2) update CUDA Toolkit (NVML is part of CUDA toolkit. An outdated or uninstalled CUDA toolkit might cause issues.)
    3) last resort: disable CodeCarbon GPU tracking by setting "gpu_ids=[]" parameter within EmissionsTracker instantiation
- ...

In [13]:
from sklearn.linear_model import LinearRegression
from codecarbon import EmissionsTracker

tracker = EmissionsTracker(project_name="CC_Demo1.1")
tracker.start()

# train the model
model = LinearRegression()
model.fit(X_train, y_train)

tracker.stop()

[codecarbon INFO @ 10:49:55] [setup] RAM Tracking...
[codecarbon INFO @ 10:49:55] [setup] GPU Tracking...
[codecarbon INFO @ 10:49:55] Tracking Nvidia GPU via pynvml


NVMLError_NotSupported: Not Supported

#### More Complex Model Training and Emissions Tracking

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from codecarbon import EmissionsTracker


model = Sequential([
        Dense(10, activation="relu", input_shape=(X_train.shape[1],)),
        Dense(10, activation="relu"),
        Dense(3, activation="softmax"),
    ])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# start tracking emissions
tracker = EmissionsTracker(project_name="CC_Demo1.2")
tracker.start()

# train the model
model.fit(X_train, y_train, epochs=100, batch_size=10)

# stop tracking emissions
tracker.stop()

[codecarbon INFO @ 14:10:35] [setup] RAM Tracking...
[codecarbon INFO @ 14:10:35] [setup] GPU Tracking...
[codecarbon INFO @ 14:10:35] Tracking Nvidia GPU via pynvml


NVMLError_NotSupported: Not Supported